In [17]:
import pandas as pd

In [18]:
sample = None

product = ['code', 'countries','url', 'product_name', 'brands', 'categories_en']

nutrient = ['energy_100g', 'fat_100g', 'saturated-fat_100g', 'carbohydrates_100g', 'sugars_100g', 'fiber_100g', 'proteins_100g', 'salt_100g', 'sodium_100g', 'polyols_100g', 'alcohol_100g']

columns_to_load = product + nutrient

opf_data = pd.read_csv('/Users/baslad01/data_dump/openfoodfacts/product_data/en.openfoodfacts.org.products.csv', sep='\t', encoding='utf-8', on_bad_lines='skip', nrows=sample,
                       usecols=columns_to_load)
opf_data.head()

/var/folders/_n/nybmwjg90rxfzcynss3bqkr80000gp/T/ipykernel_61612/1560912454.py:9: DtypeWarning: Columns (0,89,91,92) have mixed types. Specify dtype option on import or set low_memory=False.
  opf_data = pd.read_csv('/Users/baslad01/data_dump/openfoodfacts/product_data/en.openfoodfacts.org.products.csv', sep='\t', encoding='utf-8', on_bad_lines='skip', nrows=sample,


,code,url,product_name,brands,categories_en,countries,energy_100g,fat_100g,saturated-fat_100g,carbohydrates_100g,sugars_100g,polyols_100g,fiber_100g,proteins_100g,salt_100g,sodium_100g,alcohol_100g
0,00000000000000225,http://world-en.openfoodfacts.org/product/0000...,jeunes pousses,endives,"Plant-based foods and beverages,Plant-based fo...",en:france,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0000000000000207025004,http://world-en.openfoodfacts.org/product/0000...,Andrè,NaN,NaN,en:de,690.0,2.0,2.0,65.0,12.6,NaN,3.0,1.5,NaN,NaN,NaN
2,00000000000003429145,http://world-en.openfoodfacts.org/product/0000...,L.casei,NaN,NaN,Spain,NaN,1.4,0.9,9.8,9.8,NaN,NaN,2.7,0.10,0.040,NaN
3,00000000000026772226,http://world-en.openfoodfacts.org/product/0000...,Skyr,Danone,"Dairies,Fermented foods,Fermented milk product...",France,238.0,0.2,0.1,3.9,3.9,NaN,NaN,10.0,0.09,0.036,NaN
4,0000000000017,http://world-en.openfoodfacts.org/product/0000...,Vitória crackers,NaN,NaN,France,1569.0,7.0,3.08,70.1,15.0,NaN,NaN,7.8,1.40,0.560,NaN


In [23]:
# Remove the rows where the energy is missing
opf_data = opf_data[opf_data['energy_100g'].notna()]
# Remove energy if not float or int
opf_data = opf_data[opf_data['energy_100g'].apply(lambda x: isinstance(x, (int, float)))]
# Filter if countries contains france
opf_data = opf_data[opf_data['countries'].str.contains('france', case=False, na=False)]

In [21]:
def compute_calories(row, mode='simple'):
    fat = float(row['fat_100g'])
    carbohydrates = float(row['carbohydrates_100g'])
    proteins = float(row['proteins_100g'])
    polyols = float(row['polyols_100g'])
    alcohol = float(row['alcohol_100g'])
    fiber = float(row['fiber_100g'])
    # replace Nan with zeros
    fat = 0 if pd.isna(fat) else fat
    carbohydrates = 0 if pd.isna(carbohydrates) else carbohydrates
    proteins = 0 if pd.isna(proteins) else proteins
    polyols = 0 if pd.isna(polyols) else polyols
    alcohol = 0 if pd.isna(alcohol) else alcohol
    fiber = 0 if pd.isna(fiber) else fiber

    uncertain = 0.5

    if mode == 'lower':
        if fat != 0:
            fat = max(0, fat - uncertain)
        if carbohydrates != 0:
            carbohydrates = max(0, carbohydrates - uncertain)
        if proteins != 0:
            proteins = max(0, proteins - uncertain)
        if polyols != 0:
            polyols = max(0, polyols - uncertain)
        if alcohol != 0:
            alcohol = max(0, alcohol - uncertain)
        if fiber != 0:
            fiber = max(0, fiber - uncertain)
    elif mode == 'upper':
        if fat != 0:
            fat = max(0, fat + uncertain)
        if carbohydrates != 0:
            carbohydrates = max(0, carbohydrates + uncertain)
        if proteins != 0:
            proteins = max(0, proteins + uncertain)
        if polyols != 0:
            polyols = max(0, polyols + uncertain)
        if alcohol != 0:
            alcohol = max(0, alcohol + uncertain)
        if fiber != 0:
            fiber = max(0, fiber + uncertain)

    calories = 37 * fat + 17 * proteins + 29 * alcohol + 17 * (carbohydrates - polyols) + 10 * polyols + 8 * fiber
    return calories

# Compute the calories
opf_data['energy_100g_calc'] = opf_data.apply(compute_calories, axis=1)
opf_data['energy_100g_lower'] = opf_data.apply(compute_calories, axis=1, mode='lower')
opf_data['energy_100g_upper'] = opf_data.apply(compute_calories, axis=1, mode='upper')
opf_data.head()

,code,url,product_name,brands,categories_en,countries,energy_100g,fat_100g,saturated-fat_100g,carbohydrates_100g,sugars_100g,polyols_100g,fiber_100g,proteins_100g,salt_100g,sodium_100g,alcohol_100g,energy_100g_calc,energy_100g_lower,energy_100g_upper
3,00000000000026772226,http://world-en.openfoodfacts.org/product/0000...,Skyr,Danone,"Dairies,Fermented foods,Fermented milk product...",France,238.0,0.2,0.1,3.9,3.9,NaN,NaN,10.0,0.09,0.036,NaN,243.7,219.3,279.2
4,0000000000017,http://world-en.openfoodfacts.org/product/0000...,Vitória crackers,NaN,NaN,France,1569.0,7.0,3.08,70.1,15.0,NaN,NaN,7.8,1.40,0.560,NaN,1583.3,1547.8,1618.8
11,0000000000100,http://world-en.openfoodfacts.org/product/0000...,moutarde au moût de raisin,courte paille,"Condiments,Sauces,Mustards,Groceries",France,936.0,8.2,2.2,29.0,22.0,NaN,0.0,5.1,4.60,1.840,NaN,883.1,847.6,918.6
12,0000000000123,http://world-en.openfoodfacts.org/product/0000...,Sauce Sweety chili 0%,NaN,NaN,France,88.0,0.0,0.0,4.8,0.4,NaN,NaN,0.2,2.04,0.816,NaN,85.0,73.1,102.0
13,0000000000178,http://world-en.openfoodfacts.org/product/0000...,Mini coco,NaN,NaN,France,251.0,3.0,1.0,10.0,3.0,NaN,NaN,2.0,1.15,0.460,NaN,315.0,279.5,350.5


In [24]:
# Get the count of energy values not in the range
out_of_range_count = opf_data[(opf_data['energy_100g'] < opf_data['energy_100g_lower']) | (opf_data['energy_100g'] > opf_data['energy_100g_upper'])].shape[0]

# Get the total product count
total_count = opf_data.shape[0]

# Calculate the ratio
ratio = out_of_range_count / total_count

print(f"Ratio of energy not in range: {ratio:.2%}")


Ratio of energy not in range: 23.97%
